In [22]:
import pymysql  
import  pymysql.cursors 
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text 

In [43]:
db = pymysql.connect("ip","用户名","密码","库名" ) 
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
sql_2="SHOW TABLES"
cursor.execute(sql_2)
result2=cursor.fetchall()
print('result2:',result2) 
db.close()

result2: (('dim_category',), ('dim_date',), ('dim_goods',), ('dim_member',), ('dim_shop',), ('fct_sales',), ('fct_sales_item',))


In [4]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://用户名:密码@IP:端口号/库名')
sql = "SELECT * FROM fct_sales limit 8" 
df= pd.read_sql_query(sql, engine)
df.head(3)

salesID         salesNo  dimDateID  dimShopID  dimMemberID    QTY     AMT  \
0        1  34131002542176   20170602         33         4821  1.000    7.99   
1        2  34131002542177   20170602         34         1730  1.000  227.00   
2        3  34131002542178   20170602         33          513  5.046   75.64   

   pAMT   mpAMT          updateTime  
0   0.0   0.210 2017-09-09 10:24:16  
1   0.0  12.000 2017-09-09 10:24:16  
2   0.0   3.022 2017-09-09 10:24:16

In [8]:
sql = '''
select dimMemberID,sum(AMT) as money 
from dw.fct_sales where dimDateID = '20170702' and dimMemberID<>0 
group by dimMemberID 
order by sum(AMT) desc
''' 
df= pd.read_sql_query(sql, engine)
df.head(3)

dimMemberID    money
0         3110  3300.50
1         8207  2140.40
2         1677  1912.31

In [12]:
from sqlalchemy import create_engine
import pandas as pd
sql = '''select dimMemberID,sum(AMT) as money \
from dw.fct_sales \
where dimDateID = '20170702' and dimMemberID<>0 \
group by dimMemberID \
order by sum(AMT) desc'''
# read_sql_query的两个参数: sql语句， 数据库连接
df= pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
df.head(3)

dimMemberID    money
0         3110  3300.50
1         8207  2140.40
2         1677  1912.31

In [13]:
sql ="""
select dimMemberID,money
      ,case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
       end as type_money
from 
(select dimMemberID
       ,sum(AMT) as money
from dw.fct_sales 
where dimDateID = '20170702'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc ) as t; 
"""
df= pd.read_sql_query(sql, engine)
print(df_fct_sales.head(3))

   dimMemberID    money
0         3110  3300.50
1         8207  2140.40
2         1677  1912.31


In [14]:
#作业 对7月份某周每位会员累计购买金额进行分段统计，并统计每个段的会员数
#第一步 对数据探索,了解有哪些列和含义
from sqlalchemy import create_engine
import pandas as pd
sql = """SELECT * FROM fct_sales  """
df_fct_sales_total = pd.read_sql_query(sql, engine)
df_fct_sales_total.head(3) #.

salesID         salesNo  dimDateID  dimShopID  dimMemberID    QTY     AMT  \
0        1  34131002542176   20170602         33         4821  1.000    7.99   
1        2  34131002542177   20170602         34         1730  1.000  227.00   
2        3  34131002542178   20170602         33          513  5.046   75.64   

   pAMT   mpAMT          updateTime  
0   0.0   0.210 2017-09-09 10:24:16  
1   0.0  12.000 2017-09-09 10:24:16  
2   0.0   3.022 2017-09-09 10:24:16

In [16]:
# 第二步 先统计每个会员的累计购买金额  ,衍生出了 money 变量
sql ="""select dimMemberID
       ,sum(AMT) as money
from dw.fct_sales 
where dimDateID between '20170702' and '20170709'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc;"""
df = pd.read_sql_query(sql, engine)
print(df.head(6)) 

   dimMemberID     money
0         4976  11939.06
1         3110  11434.38
2         5366  11141.39
3         1677   9019.30
4          516   6487.61
5         5488   6080.59


In [17]:
# 第三步 将上一步的结果再提取，进行分频 ，，这里直接调用了衍生变量money  ,使用 case when end as
sql ="""select dimMemberID
      ,money                       
      ,case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
       end as type_money
       from 
       (select dimMemberID,sum(AMT) as money
        from dw.fct_sales 
        where dimDateID between '20170702' and '20170709'
        and dimMemberID<>0
        group by dimMemberID
        order by sum(AMT) desc) 
        as temp1
"""
df= pd.read_sql_query(sql, engine)
df.head(3) #.head(300)

dimMemberID     money type_money
0         4976  11939.06          A
1         3110  11434.38          A
2         5366  11141.39          A

In [18]:
sql ="""
select dimMemberID,count(type_money),money,type_money
from
(select dimMemberID,money,
  case when money<100 then 'D' 
       when money>=100 and money<500 then 'C'
       when money>=500 and money<1000 then 'B'
       when money>=1000  then 'A'
  end as type_money
  from 
    (select dimMemberID,sum(AMT) as money
    from dw.fct_sales 
    where dimDateID between '20170702' and '20170709'
    and dimMemberID<>0
    group by dimMemberID
    order by sum(AMT) desc) 
    as temp1
    ) 
    as temp2
    group by dimMemberID,type_money
"""
df = pd.read_sql_query(sql, engine) 
df.head(60)

dimMemberID  count(type_money)     money type_money
0          4976                  1  11939.06          A
1          3110                  1  11434.38          A
2          5366                  1  11141.39          A
3          1677                  1   9019.30          A
4           516                  1   6487.61          A
5          5488                  1   6080.59          A
6          5477                  1   5550.83          A
7          7254                  1   4548.96          A
8          3619                  1   4506.96          A
9          2103                  1   4480.00          A
10         5510                  1   4156.44          A
11         2046                  1   3225.60          A
12         1622                  1   2912.37          A
13        10078                  1   2891.72          A
14         5375                  1   2765.19          A
15         6914                  1   2658.78          A
16         4675                  1   2534.78          A
17         7156                  1   2277.50          A
18         8207                  1   2208.31          A
19         8317                  1   2099.91          A
20         8346                  1   2089.40          A
21          881                  1   2060.68          A
22         6963                  1   1972.59          A
23          293                  1   1927.72          A
24         6668                  1   1884.99          A
25         7218                  1   1837.65          A
26        10177                  1   1783.39          A
27         7269                  1   1616.90          A
28         8762                  1   1522.56          A
29          284                  1   1521.01          A
30         2653                  1   1504.06          A
31         5383                  1   1485.92          A
32          530                  1   1446.90          A
33         9562                  1   1436.02          A
34        10076                  1   1401.58          A
35         8020                  1   1388.37          A
36         7208                  1   1368.40          A
37         5402                  1   1354.90          A
38         5452                  1   1320.40          A
39         1357                  1   1279.74          A
40        10499                  1   1257.66          A
41        10493                  1   1193.34          A
42         5355                  1   1191.86          A
43         7229                  1   1184.61          A
44        10026                  1   1155.02          A
45         7054                  1   1112.70          A
46         1249                  1   1086.99          A
47         1134                  1   1080.09          A
48         2987                  1   1053.60          A
49         7786                  1   1044.00          A
50         3332                  1   1040.75          A
51         6391                  1   1033.90          A
52         7395                  1   1032.28          A
53         2985                  1   1021.20          A
54         4216                  1    982.54          B
55         5571                  1    972.87          B
56         8313                  1    957.20          B
57         1451                  1    940.50          B
58         4301                  1    939.72          B
59         6412                  1    915.40          B

In [21]:
sql ="""select count(type_money),type_money
        from
        ( select dimMemberID,money,
            case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
               end as type_money
           from 
            (select dimMemberID,sum(AMT) as money
             from dw.fct_sales 
            where dimDateID between '20170702' and '20170709'
            and dimMemberID<>0
            group by dimMemberID
            order by sum(AMT) desc) 
            as temp1
             ) 
            as temp2
           group by type_money
           """
df = pd.read_sql_query(sql, engine)
df#.head(300)

count(type_money) type_money
0                 54          A
1                 82          B
2               1385          C
3               2648          D

In [31]:
from sqlalchemy import text 
sql = text("""
SELECT COUNT(AMT) FROM fct_sales  
WHERE week(date_format(dimDateID,'%Y-%m-%d'))=28
""")
df = pd.read_sql_query(sql, engine)
print(df)  

   COUNT(AMT)
0       13299


In [34]:
#第三题  查找七月份某周订单额最大的会员的消费记录
#第一步  计算各个会员订单额
sql = """
select dimMemberID,sum(AMT) as money
from dw.fct_sales 
where dimDateID between '20170702' and '20170709'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc
limit 1"""
df = pd.read_sql_query(sql, engine)
df.head(2) 

dimMemberID     money
0         4976  11939.06

In [36]:
#第三题  查找七月份某周订单额最大的会员的消费记录
#第一步  计算各个会员订单额
sql = """
select dimMemberID
from dw.fct_sales 
where dimDateID between '20170702' and '20170709'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc
limit 1"""
df= pd.read_sql_query(sql, engine)
df.head(2)  #.head(300)

dimMemberID
0         4976

In [38]:
#第2步
sql = """
select *
from dw.fct_sales 
where dimDateID between '20170702' and '20170709' 
and dimMemberID = (select dimMemberID
                    from dw.fct_sales 
                    where dimDateID between '20170702' and '20170709'
                    and dimMemberID<>0
                    group by dimMemberID
                    order by sum(AMT) desc
                    limit 1)
            """
df = pd.read_sql_query(sql,engine)
df.head(3) 

salesID         salesNo  dimDateID  dimShopID  dimMemberID     QTY  \
0    18102  34131002560285   20170707         34         4976   1.176   
1    18867  34131002561050   20170708         34         4976  40.000   
2    49320  34131004123671   20170704         34         4976   2.000   

       AMT  pAMT    mpAMT          updateTime  
0    10.58   0.0   0.7096 2017-09-09 10:24:16  
1  1792.00   0.0  44.0001 2017-09-09 10:24:16  
2    25.00   0.0   0.8000 2017-09-09 10:24:16

In [41]:
from sqlalchemy import create_engine
import pandas as pd
sql = "SELECT * FROM dim_shop" 
df = pd.read_sql_query(sql, engine)
print(df)

   dimShopID shopCode name areaCode areaName
0         33     HD01  广场店   HDOOOO     华东大区
1         34     BJ01  花桥店   BJOOOO     北京大区
2         35     HD02  上海店   HDOOOO     华东大区
3         36   BJ0002  东城店   BJOOOO     北京大区


In [43]:
import pymysql# 打开数据库连接
db = pymysql.connect("IP","用户名",'密码',"库名" )
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 查询语句
sql = "SHOW COLUMNS FROM dim_goods"
try:
    cursor.execute(sql)
    #results = cursor.fetchone()
    results = cursor.fetchmany(6)
    #print(results)
    for i in results:
        print(i,"\t")
except:
    print ("Error: unable to fetch data")
# 关闭数据库连接
db.close()

('dimGoodsID', 'int(11)', 'NO', 'PRI', None, '') 	
('categoryID1', 'tinyint(11)', 'YES', '', None, '') 	
('categoryName1', 'varchar(255)', 'YES', '', None, '') 	
('categoryID2', 'tinyint(4)', 'YES', '', None, '') 	
('categoryName2', 'varchar(255)', 'YES', '', None, '') 	
('categoryID3', 'tinyint(4)', 'YES', '', None, '') 	


In [11]:
sql = "SELECT dimGoodsID FROM dim_goods" 
cursor.execute(sql)
results = cursor.fetchall()
print(results)

((400001,), (1010023,), (1010030,), (1010031,), (1010033,), (1010044,), (1010064,), (1010068,), (1010069,), (1010072,), (1010079,), (1010093,), (1010100,), (1010107,), (1010127,), (1010132,), (1010137,), (1010151,), (1010160,), (1010194,), (1010198,), (1010208,), (1010223,), (1010224,), (1010225,), (1010226,), (1010236,), (1010249,), (1010273,), (1010280,), (1010281,), (1010325,), (1010349,), (1010350,), (1010351,), (1010356,), (1010363,), (1010367,), (1010388,), (1010392,), (1010396,), (1010398,), (1010400,), (1010409,), (1010416,), (1010422,), (1010423,), (1010424,), (1010427,), (1010428,), (1010433,), (1010441,), (1010445,), (1010446,), (1010451,), (1010459,), (1010461,), (1010499,), (1010502,), (1010504,), (1010513,), (1110006,), (1110016,), (1110027,), (1110032,), (1110033,), (1110161,), (1110218,), (1110220,), (1110238,), (1110271,), (1110273,), (1110276,), (1110279,), (1110299,), (1110300,), (1110316,), (1110317,), (1110318,), (1110332,), (1110333,), (1110365,), (1110372,), (111

In [10]:
# 数据更新操作
#!/usr/bin/python3
import pymysql
# 打开数据库连接
db = pymysql.connect("localhost","root","deepwind","demo" )
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 更新语句
sql = "UPDATE EMPLOYEE SET AGE = AGE + 1 WHERE SEX = '%c'" % ('M')
try:
   # 执行SQL语句
   cursor.execute(sql)
   # 提交到数据库执行
   db.commit()
except:
   # 发生错误时回滚
   db.rollback()
# 关闭数据库连接
db.close()       